## It's learning of refactored model

First, we need to define weaknesses of the approach from the article:
1. Using of ResNet50 as backbone (Some more powerfull models can be used)
2. Assumption about uniform distribution of data(num of lesions)
3. Loss function and lambda parameter

What can be done with first weakness. We can change it with more strong model like SWIN Transformer (tiny) from Microsoft. That model performed better in this task compared to ResNet50. Now, transformers are performing well not only in NLP tasks, but also in CV. However, that model provide new approach(patching). Also, we can add some self-attention layers.

Second, some new approach to multi-task learning can be applied. In article they use standard hard parametr sharing, but also soft parametr sharing can be used. 

Also, X. Sun et al. proposed, <a href="https://arxiv.org/abs/1911.12423">“AdaShare: Learning What to Share for Efficient Deep Multi-Task Learning”</a> (2020). The primary goal of the researchers was to specify a single multi-layer architecture for MTL and train a policy that determines which layers to share by multiple tasks, which layers to use for specific tasks and which layers to skip for all tasks while ensuring the model provides highest performance at its most compact form.

Second weakness is not too obvious. First, we can, that distribtion of data(from EDA) is not uniform (it's assumption, but the likehood of the value being below average is higher than the likelihood of being higher). Some other distributions can be tried (Student's t-distribution, Exponential and etc), but also we can try find by use of EM algorithm (data distribution can be mixture of gaussian).

Last question is about parametr lambda in loss function. We can try to learn that parametrs. I tried one approach from <a href="https://towardsdatascience.com/multi-task-learning-with-pytorch-and-fastai-6d10dc7ce855"> here </a> and it didn't worked well. The problem is that model starts to optimize one loss and give small weights to others.

Also we cat try to expand dataset by some simple transformations (for example by albumination). From data we know, that majority of data have low bliriness(contrast). We can change it and add new images to dataset. So, it will help us to improve stability of our model.

In [ ]:
import os
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from hydra.utils import to_absolute_path

In [ ]:
GlobalHydra.instance().clear()

In [ ]:
initialize(config_path="./conf")

In [ ]:
cfg = compose(config_name="config.yaml")

In [ ]:
cfg

In [ ]:
from LDL import factory

In [ ]:
def train(cfg):

    cfg['dataset_local_train']['data_path'] = to_absolute_path(cfg['dataset_local_train']['data_path'])
    cfg['dataset_local_train']['data_file'] = to_absolute_path(cfg['dataset_local_train']['data_file'])

    cfg['dataset_local_test']['data_path'] = to_absolute_path(cfg['dataset_local_test']['data_path'])
    cfg['dataset_local_test']['data_file'] = to_absolute_path(cfg['dataset_local_test']['data_file'])

    trainer, model, train_loader, test_loader = factory.get_trainer(
        cfg['backbone']['name'],
        cfg['dataset_local_train'],
        cfg['dataset_local_test'],
        cfg['logger'],
        cfg['trainer']
    )

    trainer.fit(model, train_loader, test_loader)

    return trainer, model, train_loader, test_loader

In [ ]:
train(cfg)

Watch process of learning here https://wandb.ai/solidcake98/ImageAcneAnalysis/overview

Below I provide the picture of working of web app

<img src="./jupyter_img/TestService.png" width=1200 height=800/>